In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

## Inventory Optimization as a Flow network

**Decision Variables :**
- Quantity of items produced at each day i : $x_{i}$
- Quantity of items sold at each day i : $y_{i}$
- Quantity of items in stock at the end of each day i : $z_{i}$
- Quantity of items defered at each day i : $w_{i}$

**Objective Function :** 
Maximize $[p*y_{i} - c_{i}*x_{i} - h*z_{i} - b*w_{i}]$

**Constraints :**
1. Balance constraint : The number of products deferred equals to the number of products that are unable to be met, in other words, the quantity of demand that is larger than , so $w_{i} = d_{i} - y_{i} $.
2. Non-negative constraint :  The variable $x_{i}$, $y_{i}$, $z_{i}$ and $w_{i}$ are non-negative, so $x_{i}>=0,y_{i}>=0,z_{i}>=0,w_{i}>=0 $.
3. The initial inventory at day 1 is zero, so $z_{0}=0$, the inventory at day i (i not equals to 0) is the sum of inventory of day i-1 and quantity produced but not sold, so $z_{i}=z_{i-1} + x_{i} - y_{i}$
4. Demand constraint : the number of items sold on day i cannot exceed the demand $d_{i}$, so $y_{i}<=d_{i} $for i from 1 to T.

In [ ]:
import pyomo.environ as pyo

def inventory_optimization(p, h, b, d, c):
    # Define the index set for days
    days = range(0, len(d))

    # Create a concrete Pyomo model
    model = pyo.ConcreteModel()

    # Define the decision variables
    model.x = pyo.Var(days, within=pyo.NonNegativeIntegers) # quantity of items produced at each day i
    model.y = pyo.Var(days, within=pyo.NonNegativeIntegers) # quantity of items sold at each day i
    model.z = pyo.Var(days, within=pyo.NonNegativeIntegers) # quantity of items in stock at the end of each day i
    model.w = pyo.Var(days, within=pyo.NonNegativeIntegers) # quantity of items deferred at each day i

    # Define the objective function
    model.profit = pyo.Objective(expr=sum(p*model.y[i] - c[i]*model.x[i] - h*model.z[i] - b*model.w[i] for i in days), sense=pyo.maximize)

    # Define the balance constraints
    model.balance = pyo.ConstraintList()
    for i in days:
        if i == 0:
            # model.balance.add(model.z[0] == 0) # initial inventory is zero
            model.balance.add(model.z[i] == model.x[i] - model.y[i]) # inventory balance
        else:
            model.balance.add(model.z[i] == model.z[i-1] + model.x[i] - model.y[i]) # inventory balance

    # Define the non-negativity constraints
    model.non_negativity = pyo.ConstraintList()
    for i in days:
        model.non_negativity.add(model.x[i] >= 0)
        model.non_negativity.add(model.y[i] >= 0)
        model.non_negativity.add(model.z[i] >= 0)
        model.non_negativity.add(model.w[i] >= 0)

    # Define the demand constraints
    model.demand = pyo.ConstraintList()
    for i in days:
        model.demand.add(model.y[i] <= d[i])
        model.demand.add(model.w[i] == d[i] - model.y[i]) # balance constraint

    # Solve the model
    solver = pyo.SolverFactory('glpk')
    solver.solve(model)

    # Print the results
    print('Results:')
    print('{:<5s} {:<10s} {:<10s} {:<10s} {:<10s}'.format('Day', 'Produce', 'Sold', 'Inventory', 'Deferred'))
    for i in days:
        print('{:<5d} {:<10d} {:<10d} {:<10d} {:<10d}'.format(i, int(model.x[i]()), int(model.y[i]()), int(model.z[i]()), int(model.w[i]())))

In [ ]:
# Use the function to do the calculation
p = 8
h = 1
b = 2
d = [4, 5, 3, 6, 8, 2, 0, 6, 4, 3]
c = [1, 2, 3, 6, 8, 2, 1, 2, 5, 3]

inventory_optimization(p, h, b, d, c)

Results:
Day   Produce    Sold       Inventory  Deferred  
0     4          4          0          0         
1     5          5          0          0         
2     17         3          14         0         
3     0          6          8          0         
4     0          8          0          0         
5     2          2          0          0         
6     0          0          0          0         
7     10         6          4          0         
8     0          4          0          0         
9     3          3          0          0         
